In [1]:
import os
import time
import numpy as np
import json
import dataSorting
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import log_loss

In [2]:
np.random.seed(2020) #set seed to have reproducible results
dataRoot = 'C:/Users/Calvin/Documents/UCI/COMPSCI/175/msmd/tutorials/data_root' #specify root path of our data
train, val, test = dataSorting.loadSplitData(dataRoot) #get training, validation, test split. 60%/20%/20% respectively

In [3]:
train_spec = train[0]
train_midi = train[1]
val_spec = np.concatenate((val[0],test[0]), axis=1) #combine validation and test splits to 40% validation set(spectrogram)
val_midi = np.concatenate((val[1],test[1]), axis=1) #combine validation and test splits to 40% validation set(midi)

In [143]:
#if classes dictionary has already been created or exists, DON'T RUN THIS.
classes = dict() #create dictionary of classes. keys are the string decimal representation of binary. values are class number
class_num = 0
for idx in range(train_midi.shape[1]):
    #convert binary represenation of the class into decimal number represenation casted to string
    trueVal = str(int(''.join(map(str, np.where(train_midi[:,idx]!=0, 1, 0))), 2))
    if trueVal not in classes:
        classes[trueVal] = class_num #set the converted binary representation to be used as a key in the dictionary
        class_num += 1 #set the class number to be next available

In [144]:
#if classes dictionary has already been created or exists, DON'T RUN THIS.
js = json.dumps(classes) #save classes dictionary json into npyversion folder in data_root
f = open(dataRoot+'/npyversion/dict.json','w')
f.write(js)
f.close()

In [4]:
#run if classes dictionary file exists.
with open(dataRoot+'/npyversion/dict.json') as json_file: #load saved classes dictionary located in data_root
    classes = json.load(json_file)

In [5]:
#takes some time.
y_classed = np.empty([train_midi.shape[1], 1], dtype=int) #create empty numpy array to store ground truth classes
for idx in range(train_midi.shape[1]): #converts each midi slice into class number representation
    trueVal = str(int(''.join(map(str, np.where(train_midi[:,idx]!=0, 1, 0))), 2))
    y_classed[idx] = classes[trueVal]
y_classed = y_classed.flatten()

In [10]:
sc = StandardScaler() #create a data scaler to standardize features
sc.fit(train_spec.transpose()) #set standardization to the training split

StandardScaler(copy=True, with_mean=True, with_std=True)

In [11]:
X_train_std = sc.transform(train_spec.transpose()) #standardize training set
X_test_std = sc.transform(val_spec.transpose()) #standardize validation set

In [12]:
#DON'T RUN IF SAVED MODEL EXISTS
ppn = Perceptron(max_iter=40, eta0=0.1, random_state=0, tol=0.001) #create perceptron classifier
ppn.fit(X_train_std[:50000], y_classed[:50000]) #train perceptron classifier

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=0.1,
      fit_intercept=True, max_iter=40, n_iter=None, n_iter_no_change=5,
      n_jobs=None, penalty=None, random_state=0, shuffle=True, tol=0.001,
      validation_fraction=0.1, verbose=0, warm_start=False)

In [7]:
#run if model has been saved before.
ppn = joblib.load(dataRoot+'/npyversion/perceptronclassifier.joblib') #load classifier model.

In [13]:
y_pred = ppn.predict(X_test_std[:15000]) #predict on valiation set.

In [16]:
y_pred_bin = list() #create list to append predictions to for validation 
for p, predicted in enumerate(y_pred): #convert class to numpy array of binary to compare with ground truth midi array
    bin_str = bin(predicted)[2:] #change to binary representation of decimal number
    diff = 128-len(bin_str) #get number of padded 0's needed to reach size 128
    temp_str=''
    for i in range(diff): #pad binary numbers with 0's
        temp_str = temp_str + '0'
    bin_str = temp_str + bin_str
    y_pred_bin.append([int(z) for z in bin_str]) #append prediction
y_pred_bin = np.array(y_pred_bin).transpose()

In [19]:
for l in range(y_pred_bin.shape[1]):
    print(log_loss(val_midi[:,l], y_pred_bin[:,l], labels=[0,1])) #print loss

2.428551443251622
2.1587110058114587
3.238054015007341
1.3491896934909695
2.1587047589565356
3.507919439867198
1.8888768152262192
2.6983856338368617
1.349195940345893
2.698379386981938
2.4285389495417746
2.4285451963966986
2.9682260712770248
2.4285327026868515
3.7777536304524375
2.698379386981938
2.1587047589565356
2.1587047589565356
1.8888705683712959
2.9682198244221016
2.428538949541775
2.428538949541775
2.6983918806917853
1.888870568371296
1.8888705683712959
2.4285451963966986
3.507894452447504
2.4285451963966986
1.6190301309311326
2.4285451963966986
2.6983918806917853
1.6190301309311326
3.777716149322897
3.2380852492819585
2.158698512101612
1.8888643215163723
3.2380602618622647
1.6190301309311328
3.2380665087171883
2.158704758956535
3.2380665087171883
3.2380602618622643
2.6983918806917853
2.4285327026868515
2.9682135775671776
2.6983856338368617
1.6190301309311326
4.317422011622917
2.9682260712770248
2.1587110058114587
2.428551443251622
2.4285451963966986
3.2380602618622643
3.777753

1.3492021872008166
4.3174220116229165
4.317397024203223
1.8888768152262192
2.428538949541775
2.698379386981938
2.6983918806917853
2.6983918806917853
2.1586922652466884
2.4285451963966986
2.4285389495417755
2.4285451963966986
1.3491896934909695
2.158692265246689
2.6983918806917853
3.5079131930122744
1.8888705683712956
2.158711005811459
1.888858074661449
3.238054015007341
2.4285514432516218
3.7777348898876673
2.158711005811459
1.3491896934909697
2.428526455831928
2.4285451963966986
3.5079006993024278
2.158711005811459
3.2380602618622643
4.3174220116229165
2.968226071277025
1.6190238840762092
2.428538949541775
2.698379386981938
1.888858074661449
1.3491896934909695
3.7777473835975144
2.158698512101612
1.6190301309311328
1.6190301309311326
2.1587172526663823
0.8095150654655668
2.4285451963966986
2.4285451963966986
3.2380602618622643
2.698379386981938
0.8095150654655668
1.6190238840762092
1.6190176372212857
1.8888705683712956
2.9682260712770248
2.9682198244221016
2.158698512101612
2.42853894

1.6190301309311326
2.6983918806917853
1.8888705683712959
4.587256202208156
2.4285576901065458
2.158711005811459
1.8888768152262192
2.698379386981938
1.8888643215163723
1.6190363777860561
3.2380602618622643
2.698373140127015
2.1587172526663823
2.968219824422101
1.8888705683712956
2.158711005811459
2.9682135775671776
1.0793492560508064
1.8888705683712959
2.698379386981938
2.698379386981938
2.9682323181319488
1.6190363777860561
1.349195940345893
2.6983918806917853
1.8888705683712956
2.9682198244221016
1.888858074661449
2.4285389495417755
3.238066508717188
1.3491834466360462
2.6983856338368617
2.158711005811459
2.158723499521306
2.6983856338368617
2.428538949541775
2.158711005811459
2.698379386981938
2.1587047589565356
2.9682323181319483
2.6983856338368613
2.698379386981938
2.158711005811459
2.698379386981938
1.8888705683712959
2.158704758956535
1.8888643215163725
2.158711005811459
2.9682198244221016
3.777753630452438
2.158711005811459
2.4285389495417755
3.5079194398671985
2.69838563383686

1.8888643215163725
2.698391880691785
2.4285451963966986
2.158698512101612
2.158704758956535
1.8888705683712959
2.158711005811459
2.4285451963966986
2.9682260712770248
2.4285451963966986
2.158711005811459
1.888870568371296
1.8888705683712956
1.888858074661449
2.698379386981938
1.3491896934909695
2.1587047589565356
1.8888768152262194
1.07935550290573
2.6983918806917853
2.428538949541775
2.9682323181319483
1.8888705683712959
3.2380665087171883
1.8888705683712959
2.9682135775671776
2.158711005811459
1.8888643215163725
2.4285514432516218
1.8888705683712956
3.7777348898876673
3.507919439867198
1.6190238840762092
2.6983856338368613
1.6190301309311328
2.4285514432516218
2.6983918806917853
1.8888643215163723
1.3491896934909695
2.9682198244221016
1.8888705683712956
2.158704758956535
2.968226071277025
1.3491896934909695
4.047587821037678
4.587256202208157
2.158692265246689
2.158698512101612
2.6983856338368617
2.158711005811459
1.3491896934909695
2.4285451963966986
3.507919439867198
1.619030130931

1.8888643215163723
1.6190238840762095
2.4285327026868515
3.2380477681524176
1.07935550290573
1.6190238840762095
1.3491896934909695
1.6190238840762092
4.047587821037677
2.4285576901065458
2.9682323181319483
2.1587047589565356
1.8888705683712959
2.9682260712770248
2.158698512101612
2.1587110058114587
3.2380665087171883
2.6983918806917853
1.3492021872008166
1.6190301309311326
2.1587047589565356
1.0793492560508064
1.349195940345893
1.8888705683712959
1.888858074661449
2.158711005811459
2.1587110058114587
1.6190301309311326
1.6190301309311326
2.6983856338368617
2.158698512101612
1.07935550290573
2.4285514432516218
1.8888705683712959
2.158711005811459
2.158711005811459
3.2380602618622643
2.698373140127015
1.888864321516372
2.428538949541775
4.047562833617984
2.4285451963966986
2.4285451963966986
1.349195940345893
2.9682198244221016
1.8888768152262192
2.9682198244221016
2.4285389495417755
2.428551443251622
2.158711005811459
3.777709902467973
1.3492021872008166
1.8888643215163723
1.88886432151

...

1.6190363777860561
2.4285514432516218
2.1587172526663823
1.3492021872008166
2.4285451963966986
2.968219824422101
3.5079006993024273
1.3491834466360462
3.5078944524475038
2.428538949541775
1.8888705683712959
4.3174220116229165
2.4285514432516218
1.8888768152262192
2.4285451963966986
3.238047768152417
2.158704758956535
2.158711005811459
1.8888768152262192
2.9682260712770248
1.3491896934909695
2.6983856338368617
2.698379386981938
2.1587172526663823
1.8888705683712959
2.6983856338368617
1.8888705683712959
3.238054015007341
2.9682198244221016
4.047569080472907
2.4285514432516218
2.698379386981938
1.349195940345893
1.8888705683712956
2.968213577567178
2.9682198244221016
2.9682260712770248
2.1587047589565356
2.158711005811459
2.6983918806917853
2.4285451963966986
1.8888705683712959
2.4285451963966986
3.77772239617782
1.6190301309311326
3.2380602618622643
2.428538949541775
2.968213577567178
3.2380477681524176
1.6190301309311326
2.6983918806917853
1.6190238840762092
2.158704758956535
1.61903013

1.3491896934909697
2.428551443251622
2.428538949541775
2.9682260712770248
1.6190301309311326
3.2380665087171883
1.6190363777860561
3.2380665087171883
2.158698512101612
1.8888705683712956
2.968213577567178
3.7777348898876673
3.5079131930122744
2.9682135775671776
2.9682260712770248
2.6983731401270146
2.1587047589565356
3.238054015007341
3.238047768152417
1.6190301309311326
1.6190363777860561
1.8888768152262194
2.9682135775671776
2.698379386981938
2.4285451963966986
2.698379386981938
1.349195940345893
2.4285576901065458
2.158704758956535
2.428538949541775
2.698373140127015
2.698385633836862
1.6190301309311326
1.6190301309311328
2.158711005811459
2.1587047589565356
3.2380602618622643
2.158711005811459
4.587256202208157
2.9682198244221016
3.238054015007341
1.8888768152262192
2.9682260712770248
2.1587172526663823
2.4285451963966986
4.047587821037677
2.6983856338368617
2.4285389495417755
2.9682260712770248
2.428551443251622
2.1587047589565356
2.9682323181319483
3.5079006993024278
1.6190301309

1.8888643215163725
2.9682260712770248
1.0793492560508064
3.2380602618622643
1.6190363777860561
1.8888768152262192
2.158711005811459
3.7777348898876673
2.6983856338368617
1.349195940345893
2.4285327026868515
3.2380602618622643
2.9682198244221016
1.349195940345893
1.3491896934909695
2.6983856338368617
2.698398127546709
2.4285451963966986
1.6190301309311326
1.8888705683712956
3.2380665087171883
2.428551443251622
2.698379386981938
1.3491834466360462
1.6190301309311326
2.4285327026868515
2.6983856338368617
1.07935550290573
2.6983856338368617
2.698391880691785
2.9682135775671776
2.9682135775671776
2.4285327026868515
2.4285389495417746
2.6983918806917853
1.6190301309311328
2.9682198244221016
1.8888705683712956
2.6983918806917853
2.4285576901065458
1.8888768152262192
2.698379386981938
2.698379386981938
1.6190301309311326
4.3174220116229165
2.1587110058114587
3.238054015007341
2.698379386981938
2.6983856338368617
4.587256202208157
2.158711005811459
2.968226071277025
2.4285451963966986
3.5079006

1.8888705683712956
3.5078944524475038
1.6190301309311326
1.6190301309311328
2.4285451963966986
1.6190301309311328
2.4285451963966986
1.6190238840762092
3.5078882055925806
4.047587821037677
1.6190238840762092
2.428551443251622
0.8095150654655668
3.7777286430327437
1.8888580746614487
2.4285451963966986
1.8888705683712956
2.9682198244221016
2.9682323181319488
3.2380602618622643
2.698379386981938
2.1587047589565356
2.6983918806917853
2.158711005811459
2.4285514432516218
1.6190238840762092
2.158711005811459
2.4285451963966986
1.6190301309311328
2.1587172526663823
3.7777473835975144
1.6190301309311326
1.349195940345893
1.8888643215163725
2.4285389495417746
2.1587234995213063
1.8888768152262194
2.698373140127015
3.5079006993024273
1.8888705683712956
2.9682323181319483
3.5078882055925806
2.1587172526663823
3.2380602618622643
3.7777348898876673
2.4285451963966986
1.8888705683712959
1.3491896934909695
2.6983731401270146
1.07935550290573
2.4285451963966986
1.8888705683712959
3.238054015007341
2.1

3.2380602618622647
2.698379386981938
2.4285576901065458
2.4285451963966986
2.428551443251622
3.777716149322897
1.6190301309311326
1.349195940345893
2.1587172526663823
1.8888705683712959
2.4285389495417755
2.158698512101612
2.4285451963966986
3.2380602618622647
2.4285451963966986
1.8888705683712959
1.8888643215163723
1.8888705683712956
2.9682323181319483
2.698379386981938
1.6190238840762092
2.4285451963966986
2.4285451963966986
2.6983918806917853
2.6983918806917853
2.6983856338368617
2.158711005811459
1.6190301309311326
2.158711005811459
2.6983918806917853
1.8888705683712959
3.2380602618622643
2.428538949541775
2.158711005811459
2.9682260712770248
1.888858074661449
3.2380852492819585
2.6983856338368617
1.8888768152262192
2.9682198244221016
1.8888705683712956
2.6983918806917853
1.349195940345893
1.8888643215163723
4.047587821037677
2.1587110058114587
2.158711005811459
2.9682073307122545
2.4285514432516218
2.9682073307122545
2.1587047589565356
2.698379386981938
2.698379386981938
2.1587110

In [22]:
print(ppn.score(X_train_std[:50000], y_classed[:50000])) #print accuracy

0.21036


In [ ]:
joblib.dump(ppn, dataRoot+'/npyversion/perceptronclassifier.joblib') #save model